In [1]:
import pandas as pd
import alpaca_trade_api as tradeapi
import requests
from dotenv import load_dotenv
import os
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
load_dotenv("DLC.env")

True

In [3]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")

In [4]:
alpaca = tradeapi.REST(
    API_KEY,
    API_SECRET,
    api_version="v2")

In [5]:
tickers = ["FB", "TWTR"]

In [6]:
# As an investor the user provides us with 5 assets they want to invest in

# STEP 1 - show the investor the risk assessment of investing in these assets
# retrieve data set
# calculate percent change (daily returns)
# 1st visualization - plot daily returns, possibly plot correlatiob of assets as well
# calculate std from daily returns 
# calculate annualized std then rank assets in terms of riskiness
# calculate sharp ratios 
# 2nd visualization - plot annualized standard deviations and sharp ratios

# STEP 2 - create sample portfolios
# create 3 porfolios based on risk appetite
# 1st -> highest std , 5th -> lowest std
# $10000 investment 
# low risk portfolio - [1st - 5%, 2nd - 10%, 3rd - 15%, 4th- 20%, 5th - 50%] -- 80% investment , 20% lending
# medium risk portfolio - [1st - 10%, 2nd - 15%, 3rd - 20%, 4th- 25%, 5th - 30%] -- 60% investment , 40% lending
# high risk portfolio - [1st - 30%, 2nd - 25%, 3rd -20%, 4th- 15%, 5th - 10%] -- 50% investment , 50% lending
# ex. weights = [.05, .1, .15, .2, .5] --> daily_returns.dot(weights)

# STEP 3 - calculate cumulative returns from each portfolio
# calculate returns for each risk scenario using different weights
# calculate cumulative returns for each scenario
# show and calculate monte carlo simulation
# 3rd visualization - plot cumulative returns



In [7]:
tickers = input('what do you want to invest in?').split()

what do you want to invest in? 10000


In [8]:
capital = input('how much do you have to invest?')

how much do you have to invest? 10000


In [9]:
start = pd.Timestamp("2019-07-14", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()
timeframe = "1D"

In [10]:
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end,
    limit = 1000
).df

In [11]:
df_portfolio.head()

Empty DataFrame
Columns: [(10000, open), (10000, high), (10000, low), (10000, close), (10000, volume)]
Index: []

In [12]:
df_closing = pd.DataFrame()

In [13]:
type(df_closing)

pandas.core.frame.DataFrame

In [14]:
for ticker in tickers:
    df_closing[ticker] = df_portfolio[ticker]['close']

In [15]:
df_closing

,10000
time,


In [16]:
# STEP 1 - show the investor the risk assessment of investing in these assets
# retrieve data set
# calculate percent change (daily returns)
# 1st visualization - plot daily returns, possibly plot correlatiob of assets as well
# calculate std from daily returns 
# calculate annualized std then rank assets in terms of riskiness
# calculate sharp ratios 
# 2nd visualization - plot annualized standard deviations and sharp ratios

daily_returns = df_closing.pct_change().dropna()
daily_returns.head()

,10000
time,


In [17]:
daily_returns.plot.line(figsize=(25,10))

TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
correlations = daily_returns.corr()
sns.heatmap(correlations, vmin=-1, vmax=1)

In [ ]:
std_deviations = daily_returns.std()
annualized_std = (std_deviations * np.sqrt(252)).sort_values(ascending=False)
annualized_std

In [ ]:
sharpe_ratios = ((daily_returns.mean() * 252)) / (daily_returns.std() * np.sqrt(252))
sharpe_ratios.plot.bar()

In [ ]:
# low risk portfolio - [1st - 5%, 2nd - 10%, 3rd - 15%, 4th- 20%, 5th - 50%] -- 80% investment , 20% lending
# medium risk portfolio - [1st - 10%, 2nd - 15%, 3rd - 20%, 4th- 25%, 5th - 30%] -- 60% investment , 40% lending
# high risk portfolio - [1st - 30%, 2nd - 25%, 3rd -20%, 4th- 15%, 5th - 10%] -- 50% investment , 50% lending
# ex. weights = [.05, .1, .15, .2, .5] --> daily_returns.dot(weights)


low_risk_weights = [.05, .1, .15, .2, .5]
med_risk_weights = [.1, .15, .2, .25, .3]
high_risk_weights = [.3, .25, .2, .15, .1]
volatitly_ranks = annualized_std.index.to_list()
ordered_volatility_df = pd.concat([daily_returns[volatitly_ranks[0]], daily_returns[volatitly_ranks[1]],daily_returns[volatitly_ranks[2]],daily_returns[volatitly_ranks[3]],daily_returns[volatitly_ranks[4]]], axis=1, join='inner')
low_risk_porfolio = ordered_volatility_df.dot(low_risk_weights)
med_risk_porfolio = ordered_volatility_df.dot(med_risk_weights)
high_risk_porfolio = ordered_volatility_df.dot(high_risk_weights)

In [ ]:
ax = low_risk_porfolio.plot.line(figsize=(25,10))
med_risk_porfolio.plot.line(ax=ax)
high_risk_porfolio.plot.line(ax=ax)

In [ ]:
# STEP 3 - calculate cumulative returns from each portfolio
# calculate returns for each risk scenario using different weights
# calculate cumulative returns for each scenario
# show and calculate monte carlo simulation
# 3rd visualization - plot cumulative returns

low_risk_cumulative_returns = (1 + low_risk_porfolio).cumprod() - 1
med_risk_cumulative_returns = (1 + med_risk_porfolio).cumprod() - 1
high_risk_cumulative_returns = (1 + high_risk_porfolio).cumprod() - 1
ax = low_risk_cumulative_returns.plot()
med_risk_cumulative_returns.plot(ax=ax)
high_risk_cumulative_returns.plot(ax=ax)

In [ ]:
# Monte Carlo Simulation

